In [1]:
# set WORKDIR to the top of experiment repository
%cd ..
%pwd

/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template


/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template'

In [2]:
# import mlflow
from datetime import datetime
from sklearn.metrics import matthews_corrcoef
import pandas as pd
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
from src.settings import (
    MLFLOW_TRACKING_USERNAME,
    EXPERIMENT_NAME,
    )


def timestamp():
    """This function creates current timestamp"""
    return datetime.now().strftime("%Y_%m_%d%H_%M_%S")


# select the model for evaluation
all_models = {
    'all-mpnet-base-v2': 'sentence-transformers/all-mpnet-base-v2',
    'stsb-roberta-large': 'cross-encoder/stsb-roberta-large',
    'stsb-roberta-base': 'cross-encoder/stsb-roberta-base',
    'Legal-BERT': 'nlpaueb/legal-bert-base-uncased',
    'EURLEX-BERT': 'nlpaueb/bert-base-uncased-eurlex',
    'SciBERT': 'allenai/scibert_scivocab_uncased',
}

selected_model = all_models['Legal-BERT']



In [3]:
# load test dataset
df_test = pd.read_parquet('data/test_clean.parquet')

sentence_pairs = list(zip(df_test['text'].tolist(),df_test['text_b'].tolist()))
sentence_pairs_lds = [{"text": x[0], "text_pair": x[1]} for x in sentence_pairs]

labels_true = df_test['label'].tolist()


## Sentence-Transformers

In [4]:
from sentence_transformers.cross_encoder import CrossEncoder
import torch

# Load a pre-trained CrossEncoder model
model = CrossEncoder(selected_model, num_labels=1)

# Predict scores for a pair of sentences
scores = model.predict(sentence_pairs)
scores = model.predict(
    [
        (,),
        (,),
        (,),
    ]
)


/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able t

In [5]:
scores

array([0.52866924, 0.5696899 , 0.5411513 , 0.56135166, 0.51937073,
       0.5359785 , 0.49534476, 0.5971306 , 0.45699373, 0.4617223 ,
       0.5254464 , 0.55498594, 0.5865344 , 0.5382738 , 0.5769908 ,
       0.45926172, 0.5379857 , 0.5193613 , 0.5873433 , 0.53996336,
       0.57705367, 0.52027017, 0.6214167 , 0.58687866, 0.47739467,
       0.5033362 , 0.556111  , 0.5483626 , 0.5360353 , 0.5114348 ,
       0.54978013, 0.512177  , 0.5754094 , 0.5375439 , 0.58661795,
       0.4572795 , 0.5442641 , 0.4586027 , 0.48504   , 0.56655174,
       0.5579274 , 0.51000935, 0.58568335, 0.558887  , 0.45419514,
       0.47232154, 0.5408257 , 0.5084745 , 0.5686562 , 0.55643755,
       0.5692722 , 0.5502371 , 0.4520715 , 0.58603734, 0.5612508 ,
       0.5666476 , 0.53557783, 0.48357564, 0.49957252, 0.46137533,
       0.4696086 , 0.5496542 , 0.46850467, 0.46216702, 0.55400074,
       0.57808346, 0.53461474, 0.47380352, 0.577394  , 0.5317137 ,
       0.47649798, 0.5466648 , 0.57183444, 0.58767444, 0.56543

In [6]:
# scores to binary
# threshold = 0.45
for threshold in [
    0.53,
    0.66,
    0.75,
    0.85,
    0.90,
]:
    labels_pred = [0 if x <= threshold else 1 for x in scores]
    matthews_corrcoef_values = matthews_corrcoef(y_true=labels_true, y_pred=labels_pred)
    print(matthews_corrcoef_values)


0.017501160638229624
0.0
0.0
0.0
0.0


In [ ]:
# # save score predictions
# df_scores = pd.DataFrame(scores)
# df_scores.to_parquet('data/stsb-roberta-base_pretrain_test_scores.parquet')


## HF Transformers

### test_1

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(selected_model)
# model = AutoModelForSequenceClassification.from_pretrained(selected_model)

# scores = list()

# for sentence_pair in sentence_pairs:
#     # Example pair of legal texts
#     text1 = sentence_pair[0]
#     text2 = sentence_pair[1]

#     # Tokenize the texts
#     inputs = tokenizer(text1, text2, return_tensors='pt', truncation=True, padding=True)

#     # Get model predictions
#     outputs = model(**inputs)
#     logits = outputs.logits

#     # Get the score (e.g., similarity score)
#     score = torch.softmax(logits, dim=1)

#     scores.append(score)

### test_2

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# model = AutoModelForSequenceClassification.from_pretrained(selected_model)
# tokenizer = AutoTokenizer.from_pretrained(selected_model)

# features = tokenizer(sentence_pairs, padding=True, truncation=True, return_tensors="pt")

# model.eval()
# with torch.no_grad():
#     scores = model(**features).logits
#     print(scores)

### test_3 with pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from transformers.pipelines.text_classification import ClassificationFunction
import torch

# Check if a GPU is available and set the device
device = 0 if torch.cuda.is_available() else -1

model = AutoModelForSequenceClassification.from_pretrained(selected_model)
tokenizer = AutoTokenizer.from_pretrained(selected_model)

pipe = pipeline("text-classification", 
                model = model, 
                tokenizer = tokenizer, 
                padding = True, 
                truncation = True,
                # max_length = 512, 
                device = device, 
                function_to_apply = ClassificationFunction.SIGMOID,
                )

predictions = pipe(sentence_pairs_lds)

In [ ]:
# binarization
for threshold in [
    0.53,
    0.66,
    0.75,
    0.85,
    0.90,
]:
    labels_pred = [0 if x['score'] <= threshold else 1 for x in predictions]
    matthews_corrcoef_values = matthews_corrcoef(y_true=labels_true, y_pred=labels_pred)
    print(matthews_corrcoef_values)